In [10]:
import theano
import theano.tensor as T

import lasagne
from lasagne.layers import *

import numpy

import pickle

In [3]:
def e_relu(x):
    return T.nnet.relu(x + 1) - 1

In [2]:
inputPreds = T.tensor3("Traffic Light State", "float32")
targerSec = T.vector("Second", "int32")

predsShape = [None, 3, 303]

In [8]:
def build_net(input_shape, input_var = None):
    net = InputLayer(predsShape, inputPreds)

    net = Conv1DLayer(net, 8, 8)
    net = Conv1DLayer(net, 8, 8)
    net = Conv1DLayer(net, 8, 8)

    net = DenseLayer(net, 250)
    net = DenseLayer(net, 1, nonlinearity=e_relu)
    
    return net

In [14]:
def save_model(model, fname="finder_weights.pcl"):
    with open(fname, "wb") as f:
        pickle.dump(get_all_params(model), fname)

In [9]:
net = build_net(predsShape, inputPreds)

out = get_output(net)
params = get_all_params(net, trainable=True)

loss = T.mean((targerSec - out)**2)
updates = lasagne.updates.adam(loss, params)

train_fun = theano.function([inputPreds, targerSec], loss, updates=updates, allow_input_downcast=True)